In [2]:
from PIL import Image
import os
import random
import numpy as np
import utils
import cv2
import matplotlib.pyplot as plt

In [36]:
# Constants
REFERENCE_LOZENGE_SIZE = (331, 364)  # Our known good size for the reference image
REFERENCE_IMAGE_PATH = "data/imgs/raw/img_001.jpg"
output_dir = "data/imgs/synth"
os.makedirs(output_dir, exist_ok=True)

# Load and get dimensions of reference image
reference_img = cv2.imread(REFERENCE_IMAGE_PATH)
ref_height, ref_width = reference_img.shape[:2]
print(f"Reference image size: {ref_width}x{ref_height}")

# Calculate the reference ratio (how much of the reference image width the lozenge should occupy)
REFERENCE_WIDTH_RATIO = REFERENCE_LOZENGE_SIZE[0] / ref_width
print(f"Reference width ratio: {REFERENCE_WIDTH_RATIO:.3f}")

# Get list of lozenge variations
lozenge_dir = "data/brand_assets/processed/nectar_lozenge"
lozenge_files = [f for f in os.listdir(lozenge_dir) if f.endswith(('.png', '.PNG'))]
if not lozenge_files:
    raise ValueError("No lozenge images found in the specified directory")

# Get list of background images
background_files = [f for f in os.listdir("data/imgs/raw") if f.endswith('.jpg')][:]
padding = 20

for bg_file in background_files:
    background_path = os.path.join("data/imgs/raw", bg_file)
    background = cv2.imread(background_path)
    
    # Randomly select a lozenge for this iteration
    selected_lozenge_file = random.choice(lozenge_files)
    lozenge_path = os.path.join(lozenge_dir, selected_lozenge_file)
    lozenge = cv2.imread(lozenge_path, cv2.IMREAD_UNCHANGED)
    
    # Get background dimensions
    bg_height, bg_width = background.shape[:2]
    
    # Calculate the scaled size maintaining the same relative proportion as the reference image
    scaled_width = int(bg_width * REFERENCE_WIDTH_RATIO)
    aspect_ratio = REFERENCE_LOZENGE_SIZE[1] / REFERENCE_LOZENGE_SIZE[0]  # height/width
    scaled_height = int(scaled_width * aspect_ratio)
    
    # Scale lozenge for this specific background
    scaled_lozenge_dynamic = utils.scale_img(lozenge, (scaled_width, scaled_height))
    
    print(f"\nProcessing {bg_file}")
    print(f"Using lozenge: {selected_lozenge_file}")
    print(f"Background size: {bg_width}x{bg_height}")
    print(f"Scaled lozenge size: {scaled_lozenge_dynamic.shape}")

    # Get dimensions for placement
    loz_height, loz_width = scaled_lozenge_dynamic.shape[:2]

    # Skip images that are too small
    if bg_width < loz_width + 2*padding or bg_height < loz_height + 2*padding:
        print(f"Skipping {bg_file} - image too small for lozenge")
        continue

    regions = {
        'upper_left': {
            'x_range': (padding, min(bg_width//2 - loz_width - padding, bg_width - loz_width - padding)),
            'y_range': (padding, min(bg_height//2 - loz_height - padding, bg_height - loz_height - padding))
        },
        'center': {
            'x_range': (bg_width//4, min(3*bg_width//4 - loz_width - padding, bg_width - loz_width - padding)),
            'y_range': (bg_height//4, min(3*bg_height//4 - loz_height - padding, bg_height - loz_height - padding))
        },
        'lower_left': {
            'x_range': (padding, min(bg_width//2 - loz_width - padding, bg_width - loz_width - padding)),
            'y_range': (bg_height//2, min(bg_height - loz_height - padding, bg_height - loz_height - padding))
        }
    }

    valid_regions = {}
    for name, region in regions.items():
        x_start, x_end = region['x_range']
        y_start, y_end = region['y_range']
        
        if x_end > x_start and y_end > y_start:
            valid_regions[name] = region

    if not valid_regions:
        print(f"Skipping {bg_file} - no valid regions found")
        continue

    chosen_region = random.choice(list(valid_regions.keys()))
    region = valid_regions[chosen_region]

    x_offset = random.randint(*region['x_range'])
    y_offset = random.randint(*region['y_range'])

    print(f">> PLACING @{chosen_region} REGION, POSITION ({x_offset}, {y_offset})")

    # BLEND
    loz = scaled_lozenge_dynamic.astype(np.float32) / 255.0
    alpha_channel = loz[:, :, 3]
    rgb_loz = loz[:, :, :3]

    # Extract ROI with correct dimensions
    roi = background[y_offset:y_offset+loz_height, x_offset:x_offset+loz_width].astype(np.float32) / 255.0

    for c in range(3):
        roi[:, :, c] = roi[:, :, c] * (1 - alpha_channel) + rgb_loz[:, :, c] * alpha_channel

    roi = (roi * 255).astype(np.uint8)
    background[y_offset:y_offset+loz_height, x_offset:x_offset+loz_width] = roi

    # Save with original filename plus suffix
    output_filename = f"synth_{bg_file}"
    output_path = os.path.join(output_dir, output_filename)
    cv2.imwrite(output_path, background)

Reference image size: 1418x1417
Reference width ratio: 0.233

Processing img_158.jpg
Using lozenge: Artboard 95.png
Background size: 1080x1920
Scaled lozenge size: (277, 248, 4)
>> PLACING @lower_left REGION, POSITION (20, 1433)

Processing img_023.jpg
Using lozenge: Artboard 9_1.png
Background size: 1208x1208
Scaled lozenge size: (309, 272, 4)
>> PLACING @lower_left REGION, POSITION (260, 718)

Processing img_026.jpg
Using lozenge: Artboard 96.png
Background size: 1080x1080
Scaled lozenge size: (277, 248, 4)
>> PLACING @center REGION, POSITION (492, 277)

Processing img_202.jpg
Using lozenge: Artboard 8_1.png
Background size: 4500x4500
Scaled lozenge size: (1154, 1046, 4)
>> PLACING @lower_left REGION, POSITION (167, 2793)

Processing img_020.jpg
Using lozenge: Artboard 94.png
Background size: 1080x1080
Scaled lozenge size: (277, 248, 4)
>> PLACING @lower_left REGION, POSITION (70, 739)

Processing img_009.jpg
Using lozenge: Artboard 2_1.png
Background size: 300x600
Scaled lozenge siz

In [6]:
REFERENCE_LOZENGE_SIZE = (331, 364)  # Our known good size for the reference image
REFERENCE_IMAGE_PATH = "archive/data/imgs/raw/img_001.jpg"
output_dir = "archive/data/imgs/synth__"
os.makedirs(output_dir, exist_ok=True)

# Load and get dimensions of reference image
reference_img = cv2.imread(REFERENCE_IMAGE_PATH)
ref_height, ref_width = reference_img.shape[:2]
print(f"Reference image size: {ref_width}x{ref_height}")

# Calculate the reference ratio (how much of the reference image width the lozenge should occupy)
REFERENCE_WIDTH_RATIO = REFERENCE_LOZENGE_SIZE[0] / ref_width
print(f"Reference width ratio: {REFERENCE_WIDTH_RATIO:.3f}")

# Get list of lozenge variations
lozenge_dir = "archive/data/brand_assets/processed/nectar_lozenge"
lozenge_files = [f for f in os.listdir(lozenge_dir) if f.endswith(('.png', '.PNG'))]
if not lozenge_files:
    raise ValueError("No lozenge images found in the specified directory")

Reference image size: 1418x1417
Reference width ratio: 0.233


In [8]:
import cv2
import numpy as np
import os

# Constants
FPS = 30
DURATION = 3  # seconds
TOTAL_FRAMES = FPS * DURATION

# Paths
lozenge_dir = "archive/data/brand_assets/processed/nectar_lozenge"
reference_img_path = "archive/data/imgs/raw/img_001.jpg"  # Just for dimensions
output_path = "archive/data/imgs/synth__/lozenge_animation.mp4"

# Get reference dimensions from original image
reference_img = cv2.imread(reference_img_path)
bg_height, bg_width = reference_img.shape[:2]

# Create white background of same size
background = np.full((bg_height, bg_width, 3), 255, dtype=np.uint8)  # White background

# Rest of your code stays the same
lozenge_files = [f for f in os.listdir(lozenge_dir) if f.endswith(('.png', '.PNG'))]
selected_lozenge_file = random.choice(lozenge_files)
lozenge = cv2.imread(os.path.join(lozenge_dir, selected_lozenge_file), cv2.IMREAD_UNCHANGED)

# Scale lozenge
REFERENCE_LOZENGE_SIZE = (331, 364)
scaled_lozenge = cv2.resize(lozenge, REFERENCE_LOZENGE_SIZE)
loz_height, loz_width = scaled_lozenge.shape[:2]

# Setup video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, FPS, (bg_width, bg_height))

# Calculate movement
start_x = (bg_width - loz_width) // 2  # Center
end_x = bg_width - loz_width - 20      # Right edge with padding
total_movement = end_x - start_x

for frame_num in range(TOTAL_FRAMES):
    # Create fresh white background for each frame
    frame = background.copy()
    
    progress = frame_num / TOTAL_FRAMES
    eased_progress = (1 - np.cos(progress * np.pi)) / 2
    current_x = int(start_x + (total_movement * eased_progress))
    
    y_offset = (bg_height - loz_height) // 2
    
    # Blend lozenge
    loz = scaled_lozenge.astype(np.float32) / 255.0
    alpha_channel = loz[:, :, 3]
    rgb_loz = loz[:, :, :3]
    
    roi = frame[y_offset:y_offset+loz_height, 
                current_x:current_x+loz_width].astype(np.float32) / 255.0
    
    for c in range(3):
        roi[:, :, c] = roi[:, :, c] * (1 - alpha_channel) + rgb_loz[:, :, c] * alpha_channel
    
    roi = (roi * 255).astype(np.uint8)
    frame[y_offset:y_offset+loz_height, current_x:current_x+loz_width] = roi
    
    out.write(frame)
    print(f"Processing frame {frame_num + 1}/{TOTAL_FRAMES}")

out.release()
print("Animation complete!")

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Processing frame 1/90
Processing frame 2/90
Processing frame 3/90
Processing frame 4/90
Processing frame 5/90
Processing frame 6/90
Processing frame 7/90
Processing frame 8/90
Processing frame 9/90
Processing frame 10/90
Processing frame 11/90
Processing frame 12/90
Processing frame 13/90
Processing frame 14/90
Processing frame 15/90
Processing frame 16/90
Processing frame 17/90
Processing frame 18/90
Processing frame 19/90
Processing frame 20/90
Processing frame 21/90
Processing frame 22/90
Processing frame 23/90
Processing frame 24/90
Processing frame 25/90
Processing frame 26/90
Processing frame 27/90
Processing frame 28/90
Processing frame 29/90
Processing frame 30/90
Processing frame 31/90
Processing frame 32/90
Processing frame 33/90
Processing frame 34/90
Processing frame 35/90
Processing frame 36/90
Processing frame 37/90
Processing frame 38/90
Processing frame 39/90
Processing frame 40/90
Processing frame 41/90
Processing frame 42/90
Processing frame 43/90
Processing frame 44/

In [9]:
output_path = "archive/data/imgs/synth__/lozenge_circle.mp4"

# Constants
FPS = 30
DURATION = 30  # 30 seconds
TOTAL_FRAMES = FPS * DURATION
RADIUS = 200    # Radius of circular motion

# Get reference dimensions
reference_img = cv2.imread(reference_img_path)
bg_height, bg_width = reference_img.shape[:2]

# Create white background
background = np.full((bg_height, bg_width, 3), 255, dtype=np.uint8)

# Load and scale lozenge
lozenge_files = [f for f in os.listdir(lozenge_dir) if f.endswith(('.png', '.PNG'))]
selected_lozenge_file = random.choice(lozenge_files)
lozenge = cv2.imread(os.path.join(lozenge_dir, selected_lozenge_file), cv2.IMREAD_UNCHANGED)

REFERENCE_LOZENGE_SIZE = (331, 364)
scaled_lozenge = cv2.resize(lozenge, REFERENCE_LOZENGE_SIZE)
loz_height, loz_width = scaled_lozenge.shape[:2]

# Setup video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, FPS, (bg_width, bg_height))

# Calculate center point of the background
center_x = bg_width // 2
center_y = bg_height // 2

for frame_num in range(TOTAL_FRAMES):
    frame = background.copy()
    
    # Calculate progress (0 to 2π for a full circle)
    progress = (frame_num / TOTAL_FRAMES) * 4 * np.pi  # 2 full rotations
    
    # Determine direction based on time
    if frame_num < TOTAL_FRAMES // 2:
        # First half: clockwise
        angle = progress
    else:
        # Second half: counterclockwise
        angle = 4 * np.pi - (progress - 2 * np.pi)
    
    # Calculate position on circle
    current_x = int(center_x + RADIUS * np.cos(angle))
    current_y = int(center_y + RADIUS * np.sin(angle))
    
    # Adjust for lozenge size to keep center point on the circle
    x_offset = current_x - (loz_width // 2)
    y_offset = current_y - (loz_height // 2)
    
    # Ensure lozenge stays within frame
    x_offset = max(0, min(x_offset, bg_width - loz_width))
    y_offset = max(0, min(y_offset, bg_height - loz_height))
    
    # Blend lozenge
    loz = scaled_lozenge.astype(np.float32) / 255.0
    alpha_channel = loz[:, :, 3]
    rgb_loz = loz[:, :, :3]
    
    roi = frame[y_offset:y_offset+loz_height, 
                x_offset:x_offset+loz_width].astype(np.float32) / 255.0
    
    for c in range(3):
        roi[:, :, c] = roi[:, :, c] * (1 - alpha_channel) + rgb_loz[:, :, c] * alpha_channel
    
    roi = (roi * 255).astype(np.uint8)
    frame[y_offset:y_offset+loz_height, x_offset:x_offset+loz_width] = roi
    
    out.write(frame)
    if frame_num % FPS == 0:  # Print progress every second
        print(f"Processing second {frame_num // FPS + 1}/{DURATION}")

out.release()
print("Animation complete!")

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Processing second 1/180
Processing second 2/180
Processing second 3/180
Processing second 4/180
Processing second 5/180
Processing second 6/180
Processing second 7/180
Processing second 8/180
Processing second 9/180
Processing second 10/180
Processing second 11/180
Processing second 12/180
Processing second 13/180
Processing second 14/180
Processing second 15/180
Processing second 16/180
Processing second 17/180
Processing second 18/180
Processing second 19/180
Processing second 20/180
Processing second 21/180
Processing second 22/180
Processing second 23/180
Processing second 24/180
Processing second 25/180
Processing second 26/180
Processing second 27/180
Processing second 28/180
Processing second 29/180
Processing second 30/180
Processing second 31/180
Processing second 32/180
Processing second 33/180
Processing second 34/180
Processing second 35/180
Processing second 36/180
Processing second 37/180
Processing second 38/180
Processing second 39/180
Processing second 40/180
Processin